<a href="https://colab.research.google.com/github/BigTMiami/Erosion/blob/main/Tony_Initial_Adapter_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 1: Basic Load and Test

In [1]:
!pip install -qq adapters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.0/256.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 26.3 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer
from adapters import AutoAdapterModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoAdapterModel.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnin

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
qa_adapter = model.load_adapter("qa/squad1@ukp", config="houlsby")

(…)ter/dist/v2/index/bert-base-uncased.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

(…)/bert-base-uncased_qa_squad1_houlsby.zip:   0%|          | 0.00/7.20M [00:00<?, ?B/s]

In [4]:
model.set_active_adapters(qa_adapter)

In [5]:
from transformers import QuestionAnsweringPipeline

# Although the Hugging Face QuestionAnsweringPipeline does not officially support the BertAdapterModel, it does work when a qa header is enabled.
qa = QuestionAnsweringPipeline(model=model, tokenizer=tokenizer)

context = """
The current modus operandi in NLP involves downloading and fine-tuning pre-trained models consisting of millions or billions of parameters.
Storing and sharing such large trained models is expensive, slow, and time-consuming, which impedes progress towards more general and versatile NLP methods that learn from and for many tasks.
Adapters -- small learnt bottleneck layers inserted within each layer of a pre-trained model -- ameliorate this issue by avoiding full fine-tuning of the entire model.
However, sharing and integrating adapter layers is not straightforward.
We propose AdapterHub, a framework that allows dynamic "stitching-in" of pre-trained adapters for different tasks and languages.
The framework, built on top of the popular HuggingFace Transformers library, enables extremely easy and quick adaptations of state-of-the-art pre-trained models (e.g., BERT, RoBERTa, XLM-R) across tasks and languages.
Downloading, sharing, and training adapters is as seamless as possible using minimal changes to the training scripts and a specialized infrastructure.
Our framework enables scalable and easy access to sharing of task-specific models, particularly in low-resource scenarios.
AdapterHub includes all recent adapter architectures and can be found at AdapterHub.ml.
"""

The model 'BertAdapterModel' is not supported for . Supported models are ['AlbertForQuestionAnswering', 'BartForQuestionAnswering', 'BertForQuestionAnswering', 'BigBirdForQuestionAnswering', 'BigBirdPegasusForQuestionAnswering', 'BloomForQuestionAnswering', 'CamembertForQuestionAnswering', 'CanineForQuestionAnswering', 'ConvBertForQuestionAnswering', 'Data2VecTextForQuestionAnswering', 'DebertaForQuestionAnswering', 'DebertaV2ForQuestionAnswering', 'DistilBertForQuestionAnswering', 'ElectraForQuestionAnswering', 'ErnieForQuestionAnswering', 'ErnieMForQuestionAnswering', 'FalconForQuestionAnswering', 'FlaubertForQuestionAnsweringSimple', 'FNetForQuestionAnswering', 'FunnelForQuestionAnswering', 'GPT2ForQuestionAnswering', 'GPTNeoForQuestionAnswering', 'GPTNeoXForQuestionAnswering', 'GPTJForQuestionAnswering', 'IBertForQuestionAnswering', 'LayoutLMv2ForQuestionAnswering', 'LayoutLMv3ForQuestionAnswering', 'LEDForQuestionAnswering', 'LiltForQuestionAnswering', 'LongformerForQuestionAnswer

In [6]:
# Ignore all FutureWarnings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [7]:
def answer_questions(questions):
  for question in questions:
    result = qa(question=question, context=context)
    print("❔", question)
    print("💡", result["answer"])
    print()

answer_questions([
  "What are Adapters?",
  "What do Adapters avoid?",
  "What is proposed?",
  "What does AdapterHub allow?",
  "Where can I find AdapterHub?",
])

❔ What are Adapters?
💡 small learnt bottleneck layers inserted within each layer of a pre-trained model

❔ What do Adapters avoid?
💡 full fine-tuning of the entire model

❔ What is proposed?
💡 AdapterHub

❔ What does AdapterHub allow?
💡 dynamic "stitching-in"

❔ Where can I find AdapterHub?
💡 AdapterHub.ml



# Section 2: Sentiment Test

In [25]:
import os

import torch
from transformers import BertTokenizer
from adapters import BertAdapterModel

# Load pre-trained BERT tokenizer from Hugging Face
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# An input sentence
sentence = "It's also, clearly, great fun."

# Tokenize the input sentence and create a PyTorch input tensor
input_data = tokenizer(sentence, return_tensors="pt")

# Load pre-trained BERT model from Hugging Face Hub
# The `BertAdapterModel` class is specifically designed for working with adapters
# It can be used with different prediction heads
model = BertAdapterModel.from_pretrained('bert-base-uncased')

Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
# Load pre-trained task adapter from Adapter Hub
# This method call will also load a pre-trained classification head for the adapter task
adapter_name = model.load_adapter("sentiment/sst-2@ukp", config='pfeiffer')

# Activate the adapter we just loaded, so that it is used in every forward pass
model.set_active_adapters(adapter_name)

# Predict output tensor
outputs = model(**input_data)

print(outputs)
# Retrieve the predicted class label
# predicted = torch.argmax(outputs, dim=1).item()
# print(predicted)
# assert predicted == 1

SequenceClassifierOutput(loss=None, logits=tensor([[-4.2206,  3.9714]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [34]:
print(outputs[0])
torch.argmax(outputs[0], dim=1)

tensor([[-4.2206,  3.9714]], grad_fn=<AddmmBackward0>)


tensor([1])

In [42]:
sentences = ["I think this game is awfully fun.",
             "You think you are so good",
             "This was a pyrhic victory",
             "You are horrible!",
             "That smelled like rotten cheese"]
batch_input = tokenizer(sentences,return_tensors="pt", padding=True )
batch_output = model(**batch_input)
print(batch_output)
predicted = torch.argmax(batch_output[0], dim=1)
print(predicted)

SequenceClassifierOutput(loss=None, logits=tensor([[-3.7971,  3.4798],
        [-4.0776,  3.7608],
        [-4.1207,  3.7864],
        [ 2.8601, -2.7586],
        [ 3.5915, -3.3795]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([1, 1, 1, 0, 0])


In [40]:
batch_output[0]
torch.argmax(batch_output[0], dim=1)

tensor([1, 1, 1])

# Sentiment Test with Roberta